In [ ]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import normalize

## Dataset Configuration

In [ ]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
#d_genetic = pd.read_csv(os.path.join('data', 'dataset_genetic_cleaned_noOHE.csv'))
#d_vampire = pd.read_csv(os.path.join('data', 'dataset_vampire_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

In [ ]:
C = d_clinical.values
#G = d_genetic.values
#V = d_vampire.values

In [ ]:
y_d = outputs["dement_fail"].values
y_c = outputs["cvd_fail"].values

In [ ]:
# COMPUTATIONAL COMPLEXITY: Reduce #samples
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y_d))

In [ ]:
C_ = C[tr_idx]
C_test = C[ts_idx]
#G_ = G[tr_idx]
#G_test = G[ts_idx]
#V_ = V[tr_idx]
#V_test = V[ts_idx]

In [ ]:
y_d_ = y_d[tr_idx]
y_d_test = y_d[ts_idx]

y_c_ = y_c[tr_idx]
y_c_test = y_c[ts_idx]

In [ ]:
ds_list = [C_]#, G_, V_]
ds_test = [C_test]#, G_test, V_test]
ds_names = ['clinic']#, 'genetic', 'vampire']

## Utilities

In [ ]:
def centering_normalizing(train, test, exclusion_list = None):
    
    if exclusion_list is not None:
        scale, train = ut.centering_normalizing(train, exclusion_list)
    else:
        scale, train = ut.centering_normalizing(train)

    new_Xts = test-scale

    if exclusion_list is not None:
        new_Xts[:, exclusion_list] = test[:, exclusion_list]

    return train, new_Xts, scale


def normalizing(train, test):
    return normalize(train), normalize(test)


def make_regression(X, y, sparsity):

    scores = np.zeros(len(sparsity))
    
    for idx, sp in enumerate(sparsity):
        model = LogisticRegression(penalty = 'l1', C = sp)
        scores[idx] = np.mean(cross_validate(model,  X, y, return_train_score=False, cv=3)['test_score'])
        
    return scores[np.argmax(scores)]


def learn(C_, C_test, y, y_test, sparsity, centering = False, norm = False):
    
    if centering == True:
        C_, C_test, _ = centering_normalizing(C_, C_test)#, [5,6,7,9,10,13,15,16,17,18,19])

    if norm == True:
         C_, C_test = normalizing(C_, C_test)

    best_alpha = make_regression(C_, y, sparsity)
    model = LogisticRegression(penalty = 'l1', C = best_alpha)
    model.fit(C_, y)
    y_pred = model.predict(C_test)
    coef = model.coef_
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print("\tAccuracy: {}".format(accuracy))
    print("\tPrecision: {}".format(precision))
    print("\tRecall: {}".format(recall))
    print("Coef: {}".format(coef))
    print("BestLambda: {}".format(best_alpha))

In [ ]:
sparsity = [0.01, 0.03, 0.05, 0.07, 0.1, 0.12]

## Basic approach

### Dementia

In [ ]:
learn(C_, C_test, y_d_, y_d_test, sparsity)

### Cardio

In [ ]:
learn(C_, C_test, y_c_, y_c_test, sparsity)

## Normalized data

### Dementia

In [ ]:
learn(C_, C_test, y_d_, y_d_test, sparsity, norm = True)

### Cardio

In [ ]:
learn(C_, C_test, y_c_, y_c_test, sparsity, norm = True)

## Origin Data Centering

### Dementia

In [ ]:
learn(C_, C_test, y_d_, y_d_test, sparsity, centering = True)

### Cardio

In [ ]:
learn(C_, C_test, y_c_, y_c_test, sparsity, centering = True)

## Origin Data  Centering and Normalization

### Dementia

In [ ]:
learn(C_, C_test, y_d_, y_d_test, sparsity, centering = True, norm = True)

### Cardio

In [ ]:
learn(C_, C_test, y_c_, y_c_test, sparsity, centering = True, norm = True)